In [1]:
import keras.utils
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Embedding, Dense, Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np

In [2]:
label_encoder = LabelEncoder()
train_df = 'synthetic_train.csv'
val_df = 'synthetic_train.csv'
# Dữ liệu huấn luyện
train_data = pd.read_csv(train_df)
X_train = train_data['sentence'].values  # Cột chứa văn bản
y_train = train_data['sentiment'].values  # Cột chứa cảm xúc
# Dữ liệu xác thực
val_data = pd.read_csv(val_df)
X_val = train_data['sentence'].values  # Cột chứa văn bản
y_val = train_data['sentiment'].values  # Cột chứa cảm xúc

# Mã hóa nhãn thành dạng số
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)

In [3]:

max_words = 5000  # Số từ tối đa được xem xét
max_len = 1000  # Độ dài tối đa của một chuỗi

# Tokenize
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_train_seq = keras.utils.pad_sequences(X_train_seq, maxlen=max_len)
X_val_seq = keras.utils.pad_sequences(X_val_seq, maxlen=max_len)
embedding_dim = 120  # Kích thước của embedding cho từ
num_filters = 128  # Số lượng bộ lọc trong lớp tích chập
kernel_size = 5
num_classes = len(label_encoder.classes_)  # Số lượng lớp đầu ra
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len))
model.add(Conv1D(num_filters, kernel_size, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [4]:

batch_size = 64
epochs = 10

model.fit(X_train_seq, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val_seq, y_val))

Epoch 1/10
128/128 [==============================] - 46s 348ms/step - loss: 0.6842 - accuracy: 0.7238 - val_loss: 0.4174 - val_accuracy: 0.8195
Epoch 2/10
128/128 [==============================] - 42s 324ms/step - loss: 0.3792 - accuracy: 0.8340 - val_loss: 0.2934 - val_accuracy: 0.8847
Epoch 3/10
128/128 [==============================] - 41s 324ms/step - loss: 0.2939 - accuracy: 0.8761 - val_loss: 0.2393 - val_accuracy: 0.9082
Epoch 4/10
128/128 [==============================] - 42s 325ms/step - loss: 0.2366 - accuracy: 0.9042 - val_loss: 0.1799 - val_accuracy: 0.9401
Epoch 5/10
128/128 [==============================] - 42s 330ms/step - loss: 0.1851 - accuracy: 0.9315 - val_loss: 0.1388 - val_accuracy: 0.9571
Epoch 6/10
128/128 [==============================] - 46s 360ms/step - loss: 0.1473 - accuracy: 0.9487 - val_loss: 0.1030 - val_accuracy: 0.9740
Epoch 7/10
128/128 [==============================] - 47s 366ms/step - loss: 0.1121 - accuracy: 0.9671 - val_loss: 0.0746 - val_ac

In [5]:

loss, accuracy = model.evaluate(X_val_seq, y_val)
print(f"Mất mát trên tập xác thực: {loss}")
print(f"Độ chính xác trên tập xác thực: {accuracy}")


255/255 [==============================] - 9s 35ms/step - loss: 0.0313 - accuracy: 0.9964
Mất mát trên tập xác thực: 0.0312768928706646
Độ chính xác trên tập xác thực: 0.9964390993118286


In [6]:
y_pred = model.predict(X_val_seq)
# Convert numerical labels to original labels
y_pred_labels = np.argmax(y_pred, axis=1)
y_pred_labels = label_encoder.inverse_transform(y_pred_labels)

# Convert true labels to original labels
y_val_labels = label_encoder.inverse_transform(y_val)

# Generate classification report
report = classification_report(y_val_labels, y_pred_labels)
print(report)

255/255 [==============================] - 12s 43ms/step
              precision    recall  f1-score   support

 bình thường       0.99      1.00      0.99      2724
          tệ       1.00      1.00      1.00      2712
         tốt       1.00      0.99      0.99      2708

    accuracy                           1.00      8144
   macro avg       1.00      1.00      1.00      8144
weighted avg       1.00      1.00      1.00      8144



In [11]:
custom_text = "rất tuyệt vời"

# Tokenize and pad the custom text
custom_text_seq = tokenizer.texts_to_sequences([custom_text])
custom_text_seq = keras.utils.pad_sequences(custom_text_seq, maxlen=max_len)

# Make predictions on the custom text
custom_text_pred = model.predict(custom_text_seq)

# Convert numerical labels to original labels
custom_text_pred_label = np.argmax(custom_text_pred, axis=1)
custom_text_pred_label = label_encoder.inverse_transform(custom_text_pred_label)

# Print the predicted label
print(f"Predicted Sentiment: {custom_text_pred_label[0]}")

1/1 [==============================] - 0s 25ms/step
Predicted Sentiment: tốt


In [13]:
model.save('CNN_UIT.h5')